Loudness Range aus EBU TECH 3342

# Preprocessing

In [2]:
import librosa
import numpy as np

def preprocess_audio(audio_path, sample_rate=16000):
    """
    Vorverarbeitung einer Audiodatei: Resampling und Normalisierung.

    Args:
        audio_path (str): Pfad zur Audiodatei.
        sample_rate (int): Ziel-Sampling-Rate (Standard: 16000).

    Returns:
        np.array: Das vorverarbeitete Audio-Signal.
    """
    # Audiodatei laden und resamplen
    audio_signal, _ = librosa.load(audio_path, sr=sample_rate)

    # Zu Mono konvertieren (falls mehrkanalig)
    audio_signal = librosa.to_mono(audio_signal)

    # Normalisieren
    max_val = np.max(np.abs(audio_signal))
    if max_val > 0:
        audio_signal = audio_signal / max_val

    return audio_signal

# Funktion loudness range

In [43]:
def calculate_loudness_range(audio_signal, lower_percentile=10, upper_percentile=95):
    """
    Berechnet die Loudness Range (LRA) eines Audiosignals basierend auf EBU TECH 3342.

    Quelle:
        EBU TECH 3342: "LOUDNESS RANGE: A MEASURE TO SUPPLEMENT EBU R 128 LOUDNESS NORMALIZATION"
        Geneva, November 2023.

    Args:
        audio_signal (np.array): Das normalisierte Audio-Signal (1D-Array).
        lower_percentile (int): Unteres Perzentil (Standard: 10).
        upper_percentile (int): Oberes Perzentil (Standard: 95).

    Returns:
        float: Loudness Range (LRA) in Dezibel.
    """
    # Absolutwerte des Signals für die Perzentilberechnung
    abs_signal = np.abs(audio_signal)
    
    # Perzentile berechnen
    p_upper = np.percentile(abs_signal, upper_percentile)
    p_lower = np.percentile(abs_signal, lower_percentile)
    
    # LRA berechnen
    if p_lower > 0:  # Verhindert Division durch Null
        lra = 20 * np.log10(p_upper / p_lower)
    else:
        lra = 0.0  # Falls p_lower = 0, setzen wir die LRA auf 0 dB
    
    return lra

# Test

In [44]:
import os

# Loudness Range (LRA) für alle Dateien berechnen
for file_name in sorted(os.listdir("../audio_files")):  # Alphabetische Sortierung
    if file_name.endswith(".wav"):
        file_path = os.path.join("../audio_files", file_name)
        
        # Audiodatei vorverarbeiten
        audio_signal = preprocess_audio(file_path)
        
        # Loudness Range berechnen
        lra = calculate_loudness_range(audio_signal)
        
        print(f"Loudness Range ({file_name}): {lra:.2f} dB")

Loudness Range (_noise_pink.wav): 9.39 dB
Loudness Range (_noise_white.wav): 23.87 dB
Loudness Range (_signal_constant.wav): 0.00 dB
Loudness Range (_signal_silence.wav): 0.00 dB
Loudness Range (_signal_sine.wav): 16.08 dB
Loudness Range (_snr_03.wav): 27.89 dB
Loudness Range (_snr_10.wav): 31.94 dB
Loudness Range (_snr_20.wav): 37.36 dB
Loudness Range (example1.wav): 56.15 dB
Loudness Range (example2.wav): 54.53 dB
Loudness Range (example3.wav): 55.01 dB
